# Laboratorio 3 - Parte práctica (14 puntos)

Resolver los siguientes ítems:

1. Emplear el comando `lscpu` y en una celda de Texto indicar cual es la frecuencia del CPU en MHz.

2. Para las siguientes funciones:

   * suma de dos enteros ($a+b$).
   * resta de dos enteros ($a-b$).
   * producto de dos enteros ($a \times b$).
   * suma de los cuadrados de dos enteros ($a^2 + b^2$).
   * suma de los cubos de dos enteros ($a^3 + b^3$).
   * suma de las cuartas potencias de dos enteros ($a^4 + b^4$).

   Emplear una celda mágica para crear un archivo extensión `.c`, en otra celda compilar el archivo `.c` con las opciones vistas en la parte dirigida para que ocupe el menor espacio posible. Por último, en otra celda genere un *object file* con el mismo nombre, pero extensión `.o`, y con el comando `objdump` mostrar el código ensamblador del segmento de código del *object file* generado en el ítem anterior en sintaxis de **intel**.

3. Emplear una celda mágica para crear un archivo extensión `.c` cuyo nombre es `main`. Este archivo debe contener lo siguiente:

   * Una función que calcule la mediana de un arreglo, cuyos elementos son tipo `double`.
   * La funciones implementadas en el ítem 2.
   * Una función que devuelve el número de instrucciones de las funciones implementadas en el ítem 2. Su único argumento es un intero, si es `1` deberá devolver la cantidad de instrucciones de la función `suma`, si es `2` la cantidad de instrucciones de la función `resta`, si es `3` la cantidad de instrucciones de la función que hace el producto y así sucesivamente.
   * Una función que devuelve el CPI. La función solo debe recibir como argumentos la cantidad de ciclos, y la cantidad de instrucciones.
   * Una función que devuelve los MIPS. La función solo debe recibir como argumentos la frecuencia de reloj y el CPI.
   * Una función de nombre `CPI_mediano`. Esta función solo recibirá como argumento un número entero y deberá realizar un millón de mediciones de CPI, luego devolver la mediana de esas mediciones. El funcionamiento debe ser coherente con el tercer ítem de esta pregunta.
   * ¿Qué media pitagórica debe emplear para encontrar el CPI medio de los CPIs calculados?
   * A partir de su respuesta anterior implemente una función que permita calcular dicha media pitagórica de forma generalizada.
   * Implemente la función principal `main`. En esta función debe mostrar el CPI mediano calculado a partir de cada función, luego la media pitagórica de esas mediciones, y por último el MIPS calculado a partir de la media pitagórica de los CPIs.

4. ¿Qué opciones deberá usar para generar el ejecutable del archivo `main.c`? ¿Por qué? Genere el ejecutable con las opciones que usted ha indicado en su respuesta y muestre seis ejecuciones en celdas separadas.

Consideraciones:

* Para el ítem 2 no se pueden emplear funciones de la librería `math.h` y, en general, no puede emplear librerías auxiliares. Mostrar el código ensamblador es obligatorio, de no presentarse no se considerará la solución.
* En comentarios justifique los tipos de dato que emplea.
* El formato de presentación es un cuaderno de jupyter. No se aceptará ningún otro formato.
* Para medir los ciclos de reloj transcurridos deberá emplear la función `__rdtsc` proporcionado por la librería `x86intrin.h`. La medición de ciclos de reloj no debe estar contaminada con operaciones auxiliares, de hacerse no se considerará la solución.

Distribución del puntaje:

| Pregunta | Puntos |
|:--------:|:------:|
|     1    |   0.5  |
|     2    |   3.0  |
|    3.1   |   0.5  |
|    3.2   |   0.5  |
|    3.3   |   0.5  |
|    3.4   |   0.5  |
|    3.5   |   0.5  |
|    3.6   |   4.5  |
|    3.7   |   0.5  |
|    3.8   |   1.0  |
|    3.9   |   1.0  |
|    4.0   |   1.0  |

### Pregunta 1


Se envió un correo al profesor, indicando que opciones se puede usar para calcular la frecuencia, debido a que no cuento con Linux, y la solución fue "Adjuntar la información desde Windows en el administrador de tareas".
frecuencia = 3.30 GHz

### Pregunta 2


In [1]:
%%file lab3.c



int suma(int a, int b){

    return a + b;
}
int resta(int a, int b){
    return a - b;
}
int producto(int a, int b){
    return a * b;
}
int cuadrados(int a, int b){
    return (a*a + b*b);
}
int cubos(int a, int b){
    return ((a*a)*a + (b*b)*b);
}
int cuartas(int a, int b){
    return ((a*a)*(a*a) + (b*b)*(b*b));
}

Overwriting lab3.c


In [3]:
! gcc -c -Os lab3.c -o lab3.o 

In [5]:
! objdump -M intel -j .text -D lab3.o


lab3.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <suma>:
   0:	f3 0f 1e fa          	endbr64 
   4:	8d 04 37             	lea    eax,[rdi+rsi*1]
   7:	c3                   	ret    

0000000000000008 <resta>:
   8:	f3 0f 1e fa          	endbr64 
   c:	89 f8                	mov    eax,edi
   e:	29 f0                	sub    eax,esi
  10:	c3                   	ret    

0000000000000011 <producto>:
  11:	f3 0f 1e fa          	endbr64 
  15:	89 f8                	mov    eax,edi
  17:	0f af c6             	imul   eax,esi
  1a:	c3                   	ret    

000000000000001b <cuadrados>:
  1b:	f3 0f 1e fa          	endbr64 
  1f:	0f af ff             	imul   edi,edi
  22:	0f af f6             	imul   esi,esi
  25:	8d 04 37             	lea    eax,[rdi+rsi*1]
  28:	c3                   	ret    

0000000000000029 <cubos>:
  29:	f3 0f 1e fa          	endbr64 
  2d:	89 f8                	mov    eax,edi
  2f:	89 f2                	mov    edx,esi
  31:	0f af c7

### Pregunta 3

In [7]:
%%file main.c

#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <x86intrin.h>
#include <math.h>
// esta funcion calculará la mediana de un arreglo 
double calcular_med(long* arr, int N){
    double temp = 0.0;
    for(int i = 0; i < N - 1; i++){
        for(int j = 0; j < N - i - 1; j++){
            if (arr[j] > arr[j+1]){
                temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
    }

    if (N % 2 == 0){
        return (arr[ N / 2 ] + arr[ (N/2) - 1 ]) / 2.0;
    }
    return arr[ N / 2 ];

}
int suma(int a, int b){

    return a + b;
}
int resta(int a, int b){
    return a - b;
}
int producto(int a, int b){
    return a * b;
}
int cuadrados(int a, int b){
    return (a*a + b*b);
}
int cubos(int a, int b){
    return ((a*a)*a + (b*b)*b);
}
int cuartas(int a, int b){
    return ((a*a)*(a*a) + (b*b)*(b*b));
}

// esta funcion devolverá el número de instrucciones
// depende de que valor tome x, retornaremos el numero de instrucciones de la funcion analizada
int num_ins(int x){
    int q;
    if (x == 1){
        q = 3;
    }
    else{
        if ( x == 2){
            q = 4; 
        }
        else{
            if (x == 3){
                q = 4;
            }
            else{
                if(x == 4){
                    q = 5; 
                }
                else{
                    if(x == 5){
                        q = 9;
                    }
                    else{
                        if(x == 6){
                            q = 7;
                        }
                    }
                }
            }
        }
    }
    return q;
}

double calcular_cpi(int cant_ciclos, int cant_instrucciones){

    return ((double)cant_ciclos / cant_instrucciones);
}
double calcular_mips(double cpi, double frecuencia){

    return (frecuencia / (cpi * pow(10,6)));
}

// modulo para calcular CPI_mediano

double CPI_mediano(int num){

    srand(time(NULL));
    int a;
    int b;
    int res;

    long tic, toc;
    int N = 250000;
    long* arr = (long*)calloc(N, sizeof(long));

    for(int i = 0; i < N; i++){

        a = rand()%9;
        b = rand()%9;

        if (num == 1){
        tic = __rdtsc();
        res = suma(a,b);
        toc = __rdtsc();
        }
        else{
            if (num == 2){
                tic = __rdtsc();
                res = resta(a,b);
                toc = __rdtsc(); 
            }
            else{
                if (num == 3){
                    tic = __rdtsc();
                    res = producto(a,b);
                    toc = __rdtsc();
                }
                else{
                    if(num == 4){
                        tic = __rdtsc();
                        res = cuadrados(a,b);
                        toc = __rdtsc(); 
                    }
                    else{
                        if(num == 5){
                            tic = __rdtsc();
                            res = cubos(a,b);
                            toc = __rdtsc();
                        }
                        else{
                            if(num == 6){
                                tic = __rdtsc();
                                res = cuartas(a,b);
                                toc = __rdtsc();
                            }
                        }
                    }
                }
            }
        }


        arr[i] = (long)(toc - tic); // almaceno el número de ciclos de reloj que tomó ejecutar una instrucción específica
    }


    double mediana = calcular_med(arr,N);

    int numero_instrucciones = num_ins(num);
    double cpi_mediano = calcular_cpi(mediana,numero_instrucciones);

    return cpi_mediano;

}
// 3.7 usaremos la media aritmetica de cpi 

double media_cpi(double* arr_media, int N){

    double suma  = 0.0;

    for(int i=0; i < N; i++){

        suma  = suma + arr_media[i];

    }
    return suma/N;
}


int main(){

    double frecuencia = 3.3 * pow(10,9); //se halló en la parte 1
    
    int N = 6; //cantidad de funciones

    double* arr_media = (double*)calloc(N, sizeof(double));

    for(int i = 0; i < N; i++){

        arr_media[i] = CPI_mediano(i+1); //calculo del CPI medio de cada funcion
    }


     
    // Mostraremmos CPI mediano para cada función
    for (int i = 1; i <= N; i++) {
        double cpi_mediano = CPI_mediano(i);
        printf("CPI mediano para la función %d: %lf\n", i, cpi_mediano);
    }

    // Calculamos y mostramos la media pitagorica(media aritmetica) de los CPI medios

    for (int i = 0; i < N; i++) {
        arr_media[i] = CPI_mediano(i + 1);
    }
    double media_aritmetica_cpi = media_cpi(arr_media, N);
    printf("Media pitagorica de las mediciones: %lf\n", media_aritmetica_cpi);

    //calculo de MIPS
    double mips = calcular_mips(media_aritmetica_cpi, frecuencia);
    printf("MIPS : %lf\n", mips);


    free(arr_media);

    return 0;
}

Overwriting main.c


In [9]:
! gcc  main.c -o main -lm

In [10]:
! ./main

CPI mediano para la función 1: 11.000000
CPI mediano para la función 2: 8.250000
CPI mediano para la función 3: 8.250000
CPI mediano para la función 4: 6.600000
CPI mediano para la función 5: 3.666667
CPI mediano para la función 6: 4.714286
Media pitagorica de las mediciones: 7.080159
MIPS : 466.091245
